In [1]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import os
import torch

In [2]:
model_name='google/flan-t5-xl'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [4]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

DECODED SENTENCE:
What time is it, Tom?


In [8]:
import pandas as pd
df=pd.read_csv("/content/SongPopularity.csv")
df=df["Song"]
df.shape

(6191,)

In [28]:
def make_prompt(examples, testEx):
    prompt = ""
    for i in range(len(examples)):
        prompt += f"""(be strict) Song Name : {examples[i][0]}\n\nIs it Catchy ? {examples[i][1]}\n\n"""
    prompt += f"""Song Name : {testEx}\n\nIs it Catchy ?"""
    return prompt

In [32]:
examples = [
    ["I Need You", "Catchy"],
    ["Hurt", "Catchy"],
    ["You Take My Breath Away - Mono Version", "Not Catchy"],
    ["If I Give My Heart to You (with The Mellomen)", "Not Catchy"],
    ["Throwing It All Away - 2007 Remaster", "Not Catchy"],
    ["Does Your Chewing Gum Lose Its Flavour (On The Bedpost Overnight)", "Not Catchy"],
    ["The Rock And Roll Waltz", "Not Catchy"],
]


In [33]:
rows=df.shape
df.iloc[0]

'I Need You'

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
#tokenizer.to(device)

# Assuming df is your DataFrame
outputs = []
for i in range(rows[0]):
    testEx = df.iloc[i]
    one_shot_prompt = make_prompt(examples, testEx)
    inputs = tokenizer(one_shot_prompt, return_tensors='pt').to(device)
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=200,
        )[0],
        skip_special_tokens=True
    )
    print(f"I : {i} output : {output}")
    if output == "Yes":
        outputs.append(1)
    else:
        outputs.append(0)

Streaming output truncated to the last 5000 lines.
I : 1191 output : Yes
I : 1192 output : Yes
I : 1193 output : Yes
I : 1194 output : Not Catchy
I : 1195 output : Yes
I : 1196 output : Yes
I : 1197 output : Yes
I : 1198 output : Yes
I : 1199 output : Yes
I : 1200 output : Yes
I : 1201 output : Yes
I : 1202 output : Yes
I : 1203 output : Yes
I : 1204 output : Not Catchy
I : 1205 output : Yes
I : 1206 output : Not Catchy
I : 1207 output : Not Catchy
I : 1208 output : Yes
I : 1209 output : Yes
I : 1210 output : Yes
I : 1211 output : Yes
I : 1212 output : Yes
I : 1213 output : Yes
I : 1214 output : Not Catchy
I : 1215 output : Yes
I : 1216 output : Yes
I : 1217 output : Yes
I : 1218 output : Yes
I : 1219 output : Yes
I : 1220 output : Yes
I : 1221 output : Yes
I : 1222 output : Not Catchy
I : 1223 output : Yes
I : 1224 output : Yes
I : 1225 output : Yes
I : 1226 output : Yes
I : 1227 output : Yes
I : 1228 output : Not Catchy
I : 1229 output : Yes
I : 1230 output : Yes
I : 1231 output : Ye

In [36]:
DataFrame=pd.read_csv("/content/SongPopularity.csv")

In [37]:
DataFrame["isItCatchy"]=outputs

In [39]:
DataFrame.to_excel("Song_with_isItCatchy.xlsx")

In [41]:
DataFrame.iloc[1002]

Song                                                                Pony
Album                                       R&B: From Doo-Wop To Hip-Hop
Album Release Date                                             10/8/1999
Artist Names                                                ['Ginuwine']
Artist(s) Genres       ['hip pop', 'contemporary r&b', 'urban contemp...
Hot100 Ranking Year                                                 1996
Hot100 Rank                                                           72
Song Length(ms)                                                   251733
Spotify Link           https://open.spotify.com/track/6mz1fBdKATx6qP4...
Song Image             https://i.scdn.co/image/ab67616d00001e02980caf...
Spotify URI                         spotify:track:6mz1fBdKATx6qP4oP1I65G
Popularity                                                            74
Acousticness                                                     0.00186
Danceability                                       